In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn 
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 
from sklearn.model_selection import train_test_split , KFold , GridSearchCV
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder


In [177]:
#file_path = '/Users/user/Downloads/cybersecurity_intrusion_data.csv'
#dataset = pd.read_csv(file_path)


In [2]:
pip install ipinfo



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
'''import pandas as pd
import ipinfo

# Load the dataset
file_path = '/Users/user/Downloads/cybersecurity_intrusion_data.csv'
dataset = pd.read_csv(file_path)

# Check if the 'ip' column exists
if 'ip' not in dataset.columns:
    raise ValueError("The dataset does not contain an 'ip' column.")

# Initialize ipinfo handler (you need an access token)
access_token = 'your_ipinfo_access_token'  # Replace with your actual token
handler = ipinfo.getHandler(access_token)

# Function to extract IP features
def extract_ip_features(ip):
    try:
        details = handler.getDetails(ip)
        return {
            'country': details.country,
            'city': details.city,
            'org': details.org,
            'latitude': details.latitude,
            'longitude': details.longitude
        }
    except:
        # Handle cases where IP information cannot be fetched
        return {
            'country': 'Unknown',
            'city': 'Unknown',
            'org': 'Unknown',
            'latitude': 0.0,
            'longitude': 0.0
        }

# Extract IP features and add them to the dataset
ip_features = dataset['ip'].apply(extract_ip_features)  # Use the correct column name
ip_df = pd.DataFrame(ip_features.tolist())

# Combine the IP features with the original dataset
dataset = pd.concat([dataset, ip_df], axis=1)

# Display the updated dataset
print(dataset.head())'''

'import pandas as pd\nimport ipinfo\n\n# Load the dataset\nfile_path = \'/Users/user/Downloads/cybersecurity_intrusion_data.csv\'\ndataset = pd.read_csv(file_path)\n\n# Check if the \'ip\' column exists\nif \'ip\' not in dataset.columns:\n    raise ValueError("The dataset does not contain an \'ip\' column.")\n\n# Initialize ipinfo handler (you need an access token)\naccess_token = \'your_ipinfo_access_token\'  # Replace with your actual token\nhandler = ipinfo.getHandler(access_token)\n\n# Function to extract IP features\ndef extract_ip_features(ip):\n    try:\n        details = handler.getDetails(ip)\n        return {\n            \'country\': details.country,\n            \'city\': details.city,\n            \'org\': details.org,\n            \'latitude\': details.latitude,\n            \'longitude\': details.longitude\n        }\n    except:\n        # Handle cases where IP information cannot be fetched\n        return {\n            \'country\': \'Unknown\',\n            \'city\': 

In [178]:
file_path = '/Users/user/Downloads/cybersecurity_intrusion_data.csv'
dataset = pd.read_csv(file_path)

# Define criteria for threat levels
def assign_threat_level(row):
    if (row['login_attempts'] <= 3 and row['failed_logins'] <= 1 and row['session_duration'] <= 300):
        return 'Low'
    elif (row['login_attempts'] <= 5 and row['failed_logins'] <= 3 and row['session_duration'] <= 600):
        return 'Medium'
    else:
        return 'High'

# Add the threat_level column
dataset['threat_level'] = dataset.apply(assign_threat_level, axis=1)

# Save the updated dataset
dataset.to_csv('/Users/user/Downloads/cybersecurity_intrusion_data_with_threat_level.csv', index=False)


In [179]:
dataset.head()

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected,threat_level
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1,Medium
1,SID_00002,472,TCP,3,1557.996461,DES,0.301569,0,Firefox,0,0,High
2,SID_00003,629,TCP,3,75.044262,DES,0.739164,2,Chrome,0,1,Medium
3,SID_00004,804,UDP,4,601.248835,DES,0.123267,0,Unknown,0,1,High
4,SID_00005,453,TCP,5,532.540888,AES,0.054874,1,Firefox,0,0,Medium


In [180]:
imp = SimpleImputer(strategy='most_frequent')
dataset['encryption_used'] = imp.fit_transform(dataset[['encryption_used']]).ravel()

In [181]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9537 entries, 0 to 9536
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   session_id           9537 non-null   object 
 1   network_packet_size  9537 non-null   int64  
 2   protocol_type        9537 non-null   object 
 3   login_attempts       9537 non-null   int64  
 4   session_duration     9537 non-null   float64
 5   encryption_used      9537 non-null   object 
 6   ip_reputation_score  9537 non-null   float64
 7   failed_logins        9537 non-null   int64  
 8   browser_type         9537 non-null   object 
 9   unusual_time_access  9537 non-null   int64  
 10  attack_detected      9537 non-null   int64  
 11  threat_level         9537 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 894.2+ KB


In [182]:
original_sessionID = dataset["session_id"].unique().tolist()
original_protocol = dataset["protocol_type"].unique().tolist()
original_encryption = dataset["encryption_used"].unique().tolist()
original_browser = dataset["browser_type"].unique().tolist()
original_threat = dataset["threat_level"].unique().tolist()


In [183]:
dataset.head()

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected,threat_level
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1,Medium
1,SID_00002,472,TCP,3,1557.996461,DES,0.301569,0,Firefox,0,0,High
2,SID_00003,629,TCP,3,75.044262,DES,0.739164,2,Chrome,0,1,Medium
3,SID_00004,804,UDP,4,601.248835,DES,0.123267,0,Unknown,0,1,High
4,SID_00005,453,TCP,5,532.540888,AES,0.054874,1,Firefox,0,0,Medium


In [184]:
dataset['browser_type'].unique()

array(['Edge', 'Firefox', 'Chrome', 'Unknown', 'Safari'], dtype=object)

In [185]:
dataset = dataset.drop(['session_id'], axis = 1)

In [186]:
ore=OrdinalEncoder()
for column in dataset.columns:
    if dataset[column].dtype=='object':
        dataset[column]=ore.fit_transform(dataset[[column]])

In [187]:
dataset.head()

,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected,threat_level
0,599,1.0,4,492.983263,1.0,0.606818,1,1.0,0,1,2.0
1,472,1.0,3,1557.996461,1.0,0.301569,0,2.0,0,0,0.0
2,629,1.0,3,75.044262,1.0,0.739164,2,0.0,0,1,2.0
3,804,2.0,4,601.248835,1.0,0.123267,0,4.0,0,1,0.0
4,453,1.0,5,532.540888,0.0,0.054874,1,2.0,0,0,2.0


In [188]:
dataset.describe()

,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected,threat_level
count,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000
mean,500.430639,1.199119,4.032086,792.745312,0.300409,0.331338,1.517773,0.924819,0.149942,0.447101,0.740694
std,198.379364,0.515579,1.963012,786.560144,0.458460,0.177175,1.033988,1.189170,0.357034,0.497220,0.931130
min,64.000000,0.000000,1.000000,0.500000,0.000000,0.002497,0.000000,0.000000,0.000000,0.000000,0.000000
25%,365.000000,1.000000,3.000000,231.953006,0.000000,0.191946,1.000000,0.000000,0.000000,0.000000,0.000000
50%,499.000000,1.000000,4.000000,556.277457,0.000000,0.314778,1.000000,0.000000,0.000000,0.000000,0.000000
75%,635.000000,2.000000,5.000000,1105.380602,1.000000,0.453388,2.000000,2.000000,0.000000,1.000000,2.000000
max,1285.000000,2.000000,13.000000,7190.392213,1.000000,0.924299,5.000000,4.000000,1.000000,1.000000,2.000000


In [189]:
dataset['network_packet_size'].quantile(0.995)

1020.3199999999997

In [190]:
#min_threshold,max_threshold=dataset['session_duration'].quantile([0,0.93])


In [191]:
#dataset=dataset[(dataset.session_duration<max_threshold) & (dataset.session_duration>min_threshold)]


In [192]:
for column in dataset['session_duration']:
    first=dataset['session_duration'].quantile(0)
    second=dataset['session_duration'].quantile(0.93)
    dataset.loc[dataset['session_duration'] < first, 'session_duration'] = first
    dataset.loc[dataset['session_duration'] > second, 'session_duration'] = second

In [193]:
for column in dataset['network_packet_size']:
    firs=dataset['network_packet_size'].quantile(0)
    secon=dataset['network_packet_size'].quantile(0.995)
    dataset.loc[dataset['network_packet_size'] < firs, 'network_packet_size'] = firs
    dataset.loc[dataset['network_packet_size'] > secon, 'network_packet_size'] = secon

/var/folders/vt/p9zqs0wj0k7gdszv1qxy68v00000gn/T/ipykernel_44829/1895058609.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1020.3199999999997' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset.loc[dataset['network_packet_size'] > secon, 'network_packet_size'] = secon


In [194]:
dataset.describe()

,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected,threat_level
count,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000
mean,500.096676,1.199119,4.032086,736.339839,0.300409,0.331338,1.517773,0.924819,0.149942,0.447101,0.740694
std,197.402702,0.515579,1.963012,613.745942,0.458460,0.177175,1.033988,1.189170,0.357034,0.497220,0.931130
min,64.000000,0.000000,1.000000,0.500000,0.000000,0.002497,0.000000,0.000000,0.000000,0.000000,0.000000
25%,365.000000,1.000000,3.000000,231.953006,0.000000,0.191946,1.000000,0.000000,0.000000,0.000000,0.000000
50%,499.000000,1.000000,4.000000,556.277457,0.000000,0.314778,1.000000,0.000000,0.000000,0.000000,0.000000
75%,635.000000,2.000000,5.000000,1105.380602,1.000000,0.453388,2.000000,2.000000,0.000000,1.000000,2.000000
max,1020.000000,2.000000,13.000000,2064.881347,1.000000,0.924299,5.000000,4.000000,1.000000,1.000000,2.000000


In [195]:
dataset['ip_reputation_score'].quantile(0.995)

0.8157173991255391

In [197]:
for column in dataset['ip_reputation_score']:
    fir=dataset['ip_reputation_score'].quantile(0)
    seco=dataset['ip_reputation_score'].quantile(0.995)
    dataset.loc[dataset['ip_reputation_score'] < fir, 'ip_reputation_score'] = fir
    dataset.loc[dataset['ip_reputation_score'] > seco, 'ip_reputation_score'] = seco

In [198]:
x = dataset[['login_attempts','session_duration','failed_logins','browser_type','unusual_time_access']]
y = dataset['attack_detected']

In [199]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [200]:
def get_score(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    return model.score(x_test,y_test)

In [201]:
kf=KFold(n_splits=5)


In [202]:
score_tree =[]
for train_index,test_index in kf.split(x):
    x_train,x_test=x.iloc[train_index],x.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]
    
    score_tree.append((get_score(DecisionTreeClassifier(),x_train,x_test,y_train,y_test)))

In [203]:
score_tree

[0.7452830188679245,
 0.7295597484276729,
 0.7226009438909281,
 0.751442055584688,
 0.7420031463030938]

In [205]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize Decision Tree model
dt = DecisionTreeClassifier()

# Perform Grid Search
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [206]:
ddt = DecisionTreeClassifier(
    criterion = 'gini',
    max_depth = 5,
    max_features = None,
    min_samples_leaf = 1,
    min_samples_split = 2
)
ddt.fit(x_train,y_train)
ppd = ddt.predict(x_test)
metrics.accuracy_score(ppd,y_test)

0.8332459360251704

In [207]:
joblib.dump(ddt, 'tommy_model.pkl')
joblib.dump(original_sessionID, 'original_sessionID.pkl')
joblib.dump(original_protocol, 'original_protocol.pkl')
joblib.dump(original_encryption, 'original_encryption.pkl')
joblib.dump(original_browser, 'original_browser.pkl')
joblib.dump(original_threat, 'original_threat.pkl')

['original_threat.pkl']